In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("Libraries imported successfully!")

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [ ]:
# Create a simple RNN to analyze weight matrices
def create_analyzable_rnn(vocab_size, embedding_dim=50, hidden_units=128):
    """
    Create a simple RNN model for weight analysis
    """
    model = keras.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim, name='embedding'),
        keras.layers.SimpleRNN(hidden_units, return_sequences=True, name='rnn'),
        keras.layers.Dense(vocab_size, activation='softmax', name='output')
    ])
    return model

# Create sample data for demonstration
vocab_size = 50
sequence_length = 20
batch_size = 32

# Generate random data to initialize model
sample_data = np.random.randint(0, vocab_size, (100, sequence_length))
sample_targets = np.random.randint(0, vocab_size, (100, sequence_length))

# Create and compile model
model = create_analyzable_rnn(vocab_size, embedding_dim=30, hidden_units=64)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train for a few epochs to get meaningful weights
print("Training model briefly to initialize weights...")
model.fit(sample_data, sample_targets, epochs=5, verbose=0)

print("Model architecture:")
model.summary()

# Extract weight matrices
embedding_weights = model.get_layer('embedding').get_weights()[0]
rnn_weights = model.get_layer('rnn').get_weights()
output_weights = model.get_layer('output').get_weights()

print(f"\nWeight matrix shapes:")
print(f"Embedding weights: {embedding_weights.shape}")
print(f"RNN weights: {[w.shape for w in rnn_weights]}")
print(f"Output weights: {[w.shape for w in output_weights]}")

# For SimpleRNN, the weights are organized as:
# rnn_weights[0]: W_ih (input-to-hidden) + W_hh (hidden-to-hidden) concatenated
# rnn_weights[1]: bias
W_combined = rnn_weights[0]
bias = rnn_weights[1]

# Split the combined weight matrix
input_size = embedding_weights.shape[1]  # embedding dimension
hidden_size = W_combined.shape[1]  # hidden units

W_ih = W_combined[:input_size, :]  # Input-to-hidden
W_hh = W_combined[input_size:, :]  # Hidden-to-hidden

print(f"\nWeight matrix decomposition:")
print(f"W_ih (input-to-hidden): {W_ih.shape}")
print(f"W_hh (hidden-to-hidden): {W_hh.shape}")
print(f"Bias: {bias.shape}")

# Visualize weight matrices
plt.figure(figsize=(15, 12))

# Embedding weights
plt.subplot(3, 3, 1)
plt.imshow(embedding_weights, cmap='RdBu', aspect='auto')
plt.title('Embedding Weights')
plt.xlabel('Embedding Dimension')
plt.ylabel('Vocabulary Index')
plt.colorbar()

# Input-to-hidden weights
plt.subplot(3, 3, 2)
plt.imshow(W_ih.T, cmap='RdBu', aspect='auto')
plt.title('Input-to-Hidden Weights (W_ih)')
plt.xlabel('Input Dimension')
plt.ylabel('Hidden Unit')
plt.colorbar()

# Hidden-to-hidden weights
plt.subplot(3, 3, 3)
plt.imshow(W_hh, cmap='RdBu', aspect='auto')
plt.title('Hidden-to-Hidden Weights (W_hh)')
plt.xlabel('Hidden Unit (t-1)')
plt.ylabel('Hidden Unit (t)')
plt.colorbar()

# Weight distributions
plt.subplot(3, 3, 4)
plt.hist(embedding_weights.flatten(), bins=30, alpha=0.7, label='Embedding')
plt.hist(W_ih.flatten(), bins=30, alpha=0.7, label='W_ih')
plt.hist(W_hh.flatten(), bins=30, alpha=0.7, label='W_hh')
plt.title('Weight Distributions')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.legend()

# Weight magnitudes
plt.subplot(3, 3, 5)
weight_norms = [
    np.linalg.norm(embedding_weights),
    np.linalg.norm(W_ih),
    np.linalg.norm(W_hh)
]
weight_names = ['Embedding', 'W_ih', 'W_hh']
plt.bar(weight_names, weight_norms, color=['skyblue', 'lightcoral', 'lightgreen'])
plt.title('Weight Matrix Norms')
plt.ylabel('Frobenius Norm')
plt.xticks(rotation=45)

# Singular value decomposition of W_ih
plt.subplot(3, 3, 6)
U, s, Vt = np.linalg.svd(W_ih)
plt.plot(s, 'o-')
plt.title('Singular Values of W_ih')
plt.xlabel('Index')
plt.ylabel('Singular Value')
plt.yscale('log')
plt.grid(True, alpha=0.3)

# Rank analysis
plt.subplot(3, 3, 7)
tolerance = 1e-10
ranks = []
matrices = [embedding_weights, W_ih, W_hh]
names = ['Embedding', 'W_ih', 'W_hh']

for matrix in matrices:
    rank = np.linalg.matrix_rank(matrix, tol=tolerance)
    ranks.append(rank)

plt.bar(names, ranks, color=['skyblue', 'lightcoral', 'lightgreen'])
plt.title('Matrix Ranks')
plt.ylabel('Rank')
plt.xticks(rotation=45)

# Condition numbers
plt.subplot(3, 3, 8)
cond_numbers = []
for matrix in matrices:
    try:
        cond = np.linalg.cond(matrix)
        cond_numbers.append(cond)
    except:
        cond_numbers.append(np.inf)

plt.bar(names, cond_numbers, color=['skyblue', 'lightcoral', 'lightgreen'])
plt.title('Condition Numbers')
plt.ylabel('Condition Number')
plt.yscale('log')
plt.xticks(rotation=45)

# Weight evolution during training (conceptual)
plt.subplot(3, 3, 9)
# Simulate weight evolution
epochs = np.arange(1, 11)
w_ih_norm_evolution = weight_norms[1] * (1 + 0.1 * np.random.randn(10)).cumsum()
w_hh_norm_evolution = weight_norms[2] * (1 + 0.1 * np.random.randn(10)).cumsum()

plt.plot(epochs, w_ih_norm_evolution, 'o-', label='W_ih norm')
plt.plot(epochs, w_hh_norm_evolution, 's-', label='W_hh norm')
plt.title('Weight Evolution (Simulated)')
plt.xlabel('Epoch')
plt.ylabel('Weight Norm')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nWeight Analysis Summary:")
print(f"Embedding matrix condition number: {np.linalg.cond(embedding_weights):.2f}")
print(f"W_ih condition number: {np.linalg.cond(W_ih):.2f}")
print(f"W_hh condition number: {np.linalg.cond(W_hh):.2f}")
print(f"W_ih rank: {np.linalg.matrix_rank(W_ih)}/{min(W_ih.shape)}")
print(f"W_hh rank: {np.linalg.matrix_rank(W_hh)}/{min(W_hh.shape)}")
